In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import openpyxl
from selenium import webdriver
from selenium.webdriver.common.by import By
# https://velog.io/@sangyeon217/deprecation-warning-executablepath-has-been-deprecated
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("headless")
options.add_argument("--window-size=1920,1080")
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
#NOTE https://stackoverflow.com/questions/47316810/unable-to-locate-elements-on-webpage-with-headless-chrome
options.add_argument(f'user-agent={user_agent}')


In [113]:
#NOTE Xpath: get rid of the text() method at the end
driver = webdriver.Chrome("chromedriver.exe", options=options) 
tx_hash = "0xa6aa04a281c0d7533338530a8cfbe07d9eb21c4bfd615173297ed49728f77ce6"
driver.get(f"https://etherscan.io/tx/{tx_hash}")
element = driver.find_element(by=By.ID, value="addressCopy")
parentheses = driver.find_element(by=By.XPATH, value="//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]")
print(parentheses.text)
print(element.text)

<ipython-input-113-1be6c15fb2ed>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


0x631471b40d371648c58fa4cf5ad7caa495b59fa5
0x631471b40d371648c58fa4cf5ad7caa495b59fa5


In [3]:
filename = "Bbrick_취합파일 정리_2200617(원태님)_BSC 시작.xlsx"
workbook = openpyxl.load_workbook(filename=filename, data_only=True)
sheet = workbook["입금"]

In [5]:
#NOTE 쓸 때에는 row당 한번씩 하는게 빠르지만
#NOTE 가져올때는 한번에 가져와서 array에 담는게 좋지 않을까?
networks = sheet["L"]
# tx_hashs = sheet["S"]
for i, network in enumerate(networks):
    print(network.value)
    if i == 5:
        break

network
ERC20
ERC20
ERC20
ETH
ETH


In [140]:
for i in range(706, sheet.max_row + 1):
    network = sheet[f"L{str(i)}"].value
    if network in ["ETH", "ERC20"]:
        tx_hash = sheet[f"S{str(i)}"].value
        if "Internal" in tx_hash:
            continue 
        tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
        driver = webdriver.Chrome("chromedriver.exe", options=options) 
        driver.get(f"https://etherscan.io/tx/{tx_hash}")
        driver.implicitly_wait(0.25)
        if "error" in driver.current_url:
            sheet[f"N{str(i)}"].value = "Wrong chain or none existing tx_hash"
        else:
            # element = driver.find_element(by=By.ID, value="addressCopy")
            element = driver.find_element(by=By.XPATH, value="//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]")
            from_address = element.text
            sheet[f"N{str(i)}"].value = from_address
            
        driver.quit()
        print("row:", i, "from_address:", from_address, "txhash:", tx_hash)
        

<ipython-input-140-af17cfca7dc0>:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


row: 706 from_address: 0xafadc4302f07e9460eb4c31ec741c0f3e308ff3a txhash: 0x04fc4f1def8ac3dfb4b79939c08a6a1e697c8e3e67bf5fa66c63cbe7cc01362a
row: 707 from_address: 0x5735fbac26bb21ca3c5228022cc382136038087c txhash: 0x25c27b5a9abcce4161e06da0490725237c8a56c4f7a22a03840181b0ba3741b9
row: 709 from_address: 0x9f4745df0d6713b08323e0d39ab4cef6891c11e1 txhash: 0x668e582d055fc5bb613d7fa6c7da5ec5c1715255bd89f2fba7331f61a769d635
row: 722 from_address: 0x5735fbac26bb21ca3c5228022cc382136038087c txhash: 0xa238a5339af78e6d866bcc25464841d80fdec5f942c19f96b666b8794aedcb51
row: 726 from_address: 0x5735fbac26bb21ca3c5228022cc382136038087c txhash: 0x60c1c26e0cfdfc7b808e21797b0b6fd50fdfd17e7dc632703d8170e8bc63506a
row: 727 from_address: 0x9f4745df0d6713b08323e0d39ab4cef6891c11e1 txhash: 0xd3250ea01d9c105e9f7b5c379701b7f8989b62b6b325d3e6ff6def0df0ae1082
row: 732 from_address: 0xe3031c1bfaa7825813c562cbdcc69d96fcad2087 (Gopax: WBTC Merchant Deposit Address) txhash: 0x53bd1e7946b54f478dd7a9cd698bff7f2add29a

In [10]:

explorer_url_hash = {
    'ETH': [ ['ETH', "ETC"], "https://etherscan.io/tx/", ], 
    'BSC': [ ['BSC'], "https://bscscan.com/tx/" ] ,
}

def get_addr_of_network(target_network, start):
    global sheet
    for i in range(start, sheet.max_row + 1):
        network = sheet[f"L{str(i)}"].value
        if network in explorer_url_hash[target_network][0]:
            tx_hash = sheet[f"S{str(i)}"].value
            if "Internal" in tx_hash:
                continue 
            tx_hash = tx_hash if tx_hash[:2] == "0x" else "0x" + tx_hash
            # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            driver = webdriver.Chrome("chromedriver.exe", options=options) 
            driver.get(  explorer_url_hash[target_network][1] + tx_hash)
            driver.implicitly_wait(0.25)
            if "error" in driver.current_url:
                sheet[f"N{str(i)}"].value = "Wrong chain or none existing tx_hash"
            else:
                # element = driver.find_element(by=By.ID, value="addressCopy")
                element = driver.find_element(by=By.XPATH, value="//*[@id='ContentPlaceHolder1_maintable']/div[5]/div[2]")
                from_address = element.text
                sheet[f"N{str(i)}"].value = from_address

            driver.quit()
            print("row:", i, "from_address:", from_address, "txhash:", tx_hash)


In [11]:
get_addr_of_network('BSC', 1400)

<ipython-input-10-3a66e9a4a5ae>:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("chromedriver.exe", options=options)


row: 1400 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xf845c02fc67ef74250d6986a7c8809107cb52e5af95f294884428248376adae7
row: 1403 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xc821a87129b8c16d298e62c8ebc85fb03b7779403d7c64ed8416bf32bc6fb85d
row: 1405 from_address: 0xc8864df74a02b69992c4e680b50b9d77e13ee41f txhash: 0x687928d60cdc005e71bbdd2de53c5d74397fe3e2ab96a466c104a31fa1537e19
row: 1407 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x66f7bedd16bc0edac8370f10a60a821ec1ab86569e475950ee5ef3dd5e872b6f
row: 1409 from_address: 0x41772edd47d9ddf9ef848cdb34fe76143908c7ad (FTX Exchange) txhash: 0x41edbf7fb370af52c56d3ca1e6881739cdf09d4c43bd65abd5a064667b10ad75
row: 1412 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0x9635d1f0ceb30ac58b400b3d1d9dae8cf1c5b857d51585212d3159a9779af5b3
row: 1414 from_address: 0x8f27fa136ff4da1796368380019029d81d12d06d txhash: 0xc60674f1544500e82cb8b2d4c3618b331a223bba7b75072d9fbcf821

In [12]:
workbook.save("Bbrick_취합파일 정리_2200617(원태님)_BSC 완료.xlsx")